In [5]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Import the data
#Should change this to Input - but should reference other code to figure out what Output is
df = pd.read_csv('DF_output.csv')

# Convert ACC_DATE to datetime format
df['ACC_DATE'] = pd.to_datetime(df['ACC_DATE'], format='%Y%m%d')

# Combine ACC_DATE and ACC_TIME into a single datetime column
df['ACC_DATETIME'] = pd.to_datetime(df['ACC_DATE'].astype(str) + ' ' + df['ACC_TIME'])

# Extract hour and month from ACC_DATETIME
df['HOUR'] = df['ACC_DATETIME'].dt.hour
df['MONTH'] = df['ACC_DATETIME'].dt.month
df.to_csv('DF_Output_Processed.csv', index=False)


# Time the Operations

import time
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

start_time = time.time()

# Import the data
df = pd.read_csv('DF_output.csv')
print(f'Time taken to import data: {time.time() - start_time} seconds')

start_time = time.time()

# Convert ACC_DATE to datetime format
df['ACC_DATE'] = pd.to_datetime(df['ACC_DATE'], format='%Y%m%d')
print(f'Time taken to convert ACC_DATE: {time.time() - start_time} seconds')

start_time = time.time()

# Combine ACC_DATE and ACC_TIME into a single datetime column
df['ACC_DATETIME'] = pd.to_datetime(df['ACC_DATE'].astype(str) + ' ' + df['ACC_TIME'])
print(f'Time taken to combine ACC_DATE and ACC_TIME: {time.time() - start_time} seconds')

start_time = time.time()

# Extract hour and month from ACC_DATETIME
df['HOUR'] = df['ACC_DATETIME'].dt.hour
df['MONTH'] = df['ACC_DATETIME'].dt.month
print(f'Time taken to extract hour and month: {time.time() - start_time} seconds')

start_time = time.time()

df.to_csv('DF_Output_Processed.csv', index=False)
print(f'Time taken to write to CSV: {time.time() - start_time} seconds')

start_time = time.time()

# Pre-compute the data for the plots
hourly_counts = df.groupby(['HOUR', 'COUNTY_DESC', 'REPORT_TYPE']).size().reset_index()
hourly_counts.columns = ['HOUR', 'COUNTY_DESC', 'REPORT_TYPE', 'COUNT']
print(f'Time taken to compute hourly_counts: {time.time() - start_time} seconds')

start_time = time.time()

county_counts = df.groupby(['COUNTY_DESC', 'REPORT_TYPE']).size().unstack(fill_value=0).reset_index()
print(f'Time taken to compute county_counts: {time.time() - start_time} seconds')